# Analyzing Neural Time Series Data

### Data Extraction from MATLAB Matrix and Functions for Chapters

Written by Andrew J. Graves on 12/03/19

In [1]:
# Import modules
from scipy import io
from scipy import signal as sig
from math import fabs
import numpy as np
import mne
#print(mne.__version__)

### Import data

In [2]:
# Load Sample MATLAB/EEGLAB object from current directory
EEG = io.loadmat('sampleEEGdata.mat')['EEG']

# Get the data and reshape data from 3D (chan x time x trial) to 2D (chan x (time x trial))
eeg_data = EEG['data'].item()
two_d = eeg_data.reshape(eeg_data.shape[0], (eeg_data.shape[1]*eeg_data.shape[2]), order='F')

# Extract sampling rate, channel names, channel types, and time
samp_rate = EEG['srate'].item()
chan_names = sum(np.concatenate(EEG['chanlocs'].item()['labels'].tolist()).tolist(), [])
chan_types = ['eeg'] * len(chan_names)
eeg_time = EEG['times'].item()[0]

# Create an MNE info object and combine 2D data with info for an MNE raw object
mont = mne.channels.make_standard_montage('biosemi64')
info = mne.create_info(ch_names=chan_names, sfreq=samp_rate, ch_types=chan_types, montage=mont)
raw = mne.io.RawArray(two_d, info, verbose=False)

# Extract various features useful for plots
epochs = EEG['epoch'].item()[0]
chan_theta = EEG['chanlocs'].item()['theta']
chan_radius = EEG['chanlocs'].item()['radius'].tolist()

The following cells are an attempt to "modularize" some of Cohen's codes, by compacting them into re-useable functions. These instantiations of pre-processing and analysis steps are directly translated from Cohen's code, and HAVE NOT been rigorously tested. The functions help reduce error for performing the same computation multiple times, particularly with the difference between MATLAB and Python types / indexing.

#### Chapter 9

In [3]:
# ERP
def get_erp(three_d, chan_name):
    """Returns an event-related potential at a specific channel
    
    Parameters
    ----------
    three_d: numpy array
        A 3-dimensional EEG data structure (channel x time x trial)
    
    chan_name: str
        The name of the channel to analyze
    
    Returns
    -------
    erp: ndarray
        An average voltage averaged across every trial at a specific channel
    """
    chan_index = [index for index, item in enumerate(chan_names) if item == chan_name]
    erp = np.squeeze(np.mean(three_d[list(chan_index), :, :], 2))
    return(erp)

# Low-pass filter for ERPs
def low_pass_erp(erp, filter_cutoff, trans_width, nyq):
    """Returns a low-pass filtered event-related potential
    
    Parameters
    ----------
    erp: numpy array
        A 1-dimensional event-related potential data structure
    
    filter_cutoff: int or float
        The threshold value for high frequencies to filter out
        
    trans_width: int or float
        The transition width of the filter
    
    nyq: int or float
        The nyquist frequency
    
    Returns
    -------
    low_pass: ndarray
        A low-pass filtered event-related potential
    """
    nyq = int(nyq)
    f_freqs = np.array([0, filter_cutoff, filter_cutoff * (1 + trans_width), nyq]) / nyq
    ideal_resp = np.array([1, 1, 0, 0])
    filter_weights = sig.firls(101, f_freqs, ideal_resp)
    low_pass = sig.filtfilt(filter_weights, 1, erp)
    return(low_pass)

# Band-pass filter for ERPs
def band_pass_erp(erp, filter_low, filter_high, trans_width, nyq):
    """Returns a band-pass filtered event-related potential
    
    Parameters
    ----------
    erp: numpy array
        A 1-dimensional event-related potential data structure
    
    filter_low: int or float
        The threshold value for low frequencies to filter out
        
    filter_high: int or float
        The threshold value for high frequencies to filter out
        
    trans_width: int or float
        The transition width of the filter
    
    nyq: int or float
        The nyquist frequency
    
    Returns
    -------
    band_pass: ndarray
        A band-pass filtered event-related potential
    """
    nyq = int(nyq)
    f_freqs = np.array([0, filter_low * (1 - trans_width), filter_low, 
                        filter_high, filter_high * (1 + trans_width), nyq]) / nyq
    ideal_resp = np.array([0, 0, 1, 1, 0, 0])
    filter_weights = sig.firls(round(3 * (nyq * 2 / filter_low) + 1), f_freqs, ideal_resp)
    band_pass = sig.filtfilt(filter_weights, 1, erp)
    return(band_pass)

#### Chapter 11

In [4]:
# Generate a time series vector
def time_series(srate, start_time=-1, end_time=1):
    """Returns a time series vector
       Parameters
    ----------
    srate: int or float
        Sampling rate of the signal in Hz
    
    start_time: int or float
        Start time of the signal. Defaults to -1
        
    end_time: int or float
        End time of the signal. Defaults to 1
    
    Returns
    -------
    time_series: ndarray
        The time series vector
    """
    time_series = np.arange(start_time, end_time + 1 / srate, 1 / srate)
    return time_series

# Generate a sine wave
def get_sine_wave(frequency, srate, amplitude=1, start_time=-1, end_time=1, phase=0, cos=False, complex_wave=False):
    """Returns a simulated sine wave
    
    Parameters
    ----------
    frequency: int or float
        Frequency of the sine wave
        
    srate: int or float
        Sampling rate of the signal in Hz
        
    amplitude: int or float
        Amplitude of the sine wave. Defaults to 1
    
    start_time: int or float
        Start time of the signal. Defaults to -1
        
    end_time: int or float
        End time of the signal. Defaults to 1
        
    phase: int or float
        The phase angle offset of the sine wave. Defaults to 0
        
    cos: bool
        If True, generate a cosine wave instead of a sine wave. Defaults to False
    
    complex_wave: bool
        If True, generate a complex sine wave instead of a real sine wave. Defaults to False
    
    Returns
    -------
    sine_wave: ndarray
        The simulated sine wave
    """
    time = np.arange(start_time, end_time + 1 / srate, 1 / srate)
    if cos == True:
        sine_wave = amplitude * np.cos(2 * np.pi * frequency * time + phase)
    elif complex_wave == True:
        sine_wave = np.exp(2 * 1j * np.pi * frequency * time + phase)
    else: 
        sine_wave = amplitude * np.sin(2 * np.pi * frequency * time + phase)
    return sine_wave

# Compute power with numpy fourier transform
def compute_power(signal, srate):
    """Compute power with numpy fourier transform. Note that Cohen multiplied the series by 2 in the code 
       rather than squaring it for some reason. AFAIK squaring is appropriate, which is why the magnitudes 
       of my frequencies are larger
    
    Parameters
    ----------
    signal: ndarray
        A 1-dimensional signal array
    
    srate: int or float
        The sampling rate of the signal
    
    Returns
    -------
    tuple
        power: ndarray
            The power at particular frequencies
        frequencies: ndarray
            The frequency index for each power entry
    """
    frequencies = np.linspace(0, int(srate / 2), int(len(signal) / 2) + 1)
    fft_sig = np.fft.fft(signal) / len(signal)
    power = np.abs(fft_sig[0:len(frequencies)]) ** 2
    return power, frequencies

# Compute phase with numpy fourier transform
def compute_phase(signal, srate):
    """Compute phase with numpy fourier transform.
    
    Parameters
    ----------
    signal: ndarray
        A 1-dimensional signal array
    
    srate: int or float
        Sampling rate of the signal in Hz
    
    Returns
    -------
    tuple
        phase: ndarray
            The phase at particular frequencies
        frequencies: ndarray
            The frequency index for each phase entry
    """
    frequencies = np.linspace(0, int(srate / 2), int(len(signal) / 2) + 1)
    fft_sig = np.fft.fft(signal) / len(signal)
    phase = np.angle(fft_sig[0:len(frequencies)], deg=False)
    return phase, frequencies

# Replicate MATLABS conv 'same' method
def matlab_conv_same(signal, kernel):
    """Directly replicate MATLAB's conv(kernel, signal, 'same') method
    
    Parameters
    ----------
    signal: ndarray
        A 1-dimensional signal array
    
    kernel: ndarray
        The kernel to convolve with the signal
    
    Returns
    -------
    conv_res: ndarray
        The signal convolved with the kernel
    """
    n_pad = len(kernel) - 1
    full = np.convolve(signal, kernel, 'full')
    first = n_pad - n_pad // 2
    conv_res = full[first:first + len(signal)]
    return conv_res

# Convert integer into alphabet character
def pos_to_char(pos):
    """Convert position in the alphabet to its associated character. Useful for iterative labelling in plots
    
    Parameters
    ----------
    pos: int
        The index of the desired alphabet character
    
    Returns
    -------
    char: str
        The alphabet character from the input index
    """
    char = chr(pos + 97).upper()
    return char

#### Chapter 12

In [5]:
# Generate a Gaussian window for Morlet wavelets
def gauss_win(frequency, srate, num_cycle, start_time=-1, end_time=1, normalize=False):
    """Generate a Gaussian window for Morlet wavelets
    
    Parameters
    ----------
    frequency: int or float
        Frequency of the signal in Hz
    
    srate: int or float
        Sampling rate of the signal in Hz
        
    num_cycle: int or float
        The number of wavelet cycles; numerator term for computing the standard deviation
        
    start_time: int or float
        Start time of the signal. Defaults to -1
        
    end_time: int or float
        End time of the signal. Defaults to 1
        
    normalize: bool
        If True, normalizes the wavelet by the frequency. Defaults to False
    
    Returns
    -------
    gaussian_window: ndarray
        The Gaussian window for Morlet wavelet transformation
    """
    time = np.arange(start_time, end_time + 1 / srate, 1 / srate)
    sd = num_cycle / (2 * np.pi * frequency)
    if normalize == False:
        gaussian_window = np.exp((-time ** 2) / (2 * sd ** 2))
    else: 
        gaussian_window = np.exp((-time ** 2) / (2 * sd ** 2)) / frequency
    return gaussian_window

#### Chapter 13

In [6]:
# Similar to MATLAB's dsearchn
def find_nearest(array, values):
    """Similar to MATLAB's dsearchn. Finds indices of values closest to the input array
    
    Parameters
    ----------
    array: list or ndarray
        The array to search through
    
    values: list
        The values to find the nearest neighbors of in the input array
    
    Returns
    -------
    indices: list
        The index of the nearest neighbors from the input array
    """
    indices = []
    for i in range(len(values)):
        idx = np.searchsorted(array, values[i], side="left")
        if idx > 0 and (idx == len(array) or fabs(values[i] - array[idx - 1]) < fabs(values[i] - array[idx])):
            indices.append(idx - 1)
        else:
            indices.append(idx)
    return indices